In [1]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from magic_game import check_answer, bit_to_num
from quantum_model import Quantum_Strategy

In [24]:
n = 5
data = np.load(f"data_{n}.npz")
data_size = len(data['X'])
x_train = data['X']; y_train = data['Y']
x_test = np.random.randint(0, 2, (1000, x_train.shape[1]))
# one-hot
x_train = keras.utils.to_categorical(x_train, 2)
x_test = keras.utils.to_categorical(x_test, 2)
# 0, 1, start=2, end=3
y_train = np.hstack([np.ones((data_size, 1)) * 2, y_train, np.ones((data_size, 1)) * 3])
y_train = keras.utils.to_categorical(y_train, 4)
decoder_input_data = y_train[:, :-1]
decoder_target_data = y_train[:, 1:]
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
max_decoder_seq_length = decoder_input_data.shape[1]

x_train shape: (10000, 20, 2)
y_train shape: (10000, 22, 4)
10000 train samples
1000 test samples


In [52]:
latent_dim = 256
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, 2))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, 4))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(4, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [53]:
model.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, None, 2)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, None, 4)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_10 (LSTM)      │ [(None, 256),     │    265,216 │ input_layer_18[0… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ [(None, None,     │    267,264 │ input_layer_19[0… │
│                     │ 256), (None,      │            │ lstm_10[0][1],    │
│                     │ 256), (None,      │            │ lstm_10[0][2]     │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 4)   │      1,028 │ lstm_11[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 533,508 (2.04 MB)

 Trainable params: 533,508 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
)

In [55]:
batch_size = 1000
epochs = 100

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model.keras", save_best_only=True),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=10),
]

model.fit(
    [x_train, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
)

Epoch 1/100


9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 700ms/step - loss: 1.2165 - val_loss: 0.8307
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 735ms/step - loss: 0.8250 - val_loss: 0.7972
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 676ms/step - loss: 0.7833 - val_loss: 0.7494
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 677ms/step - loss: 0.7389 - val_loss: 0.7068
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 714ms/step - loss: 0.7039 - val_loss: 0.6891
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 698ms/step - loss: 0.6871 - val_loss: 0.6780
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 695ms/step - loss: 0.6755 - val_loss: 0.6653
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 726ms/step - loss: 0.6656 - val_loss: 0.6627
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 716ms/step - loss: 0.6626 - val_loss: 0.6624
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 694ms/step - loss: 0.6617 - val_loss: 0.6608
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 697ms/step - loss: 0.6612 - val_loss: 0.6612
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 677ms/step - loss: 0.6612 - val_lo

KeyboardInterrupt: 

In [56]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 4))
    # Populate the first character of target sequence with the start character 2.
    target_seq[0, 0, 2] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_char = np.argmax(output_tokens[0, -1, :])
        decoded_sentence += [sampled_char]


        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == 3 or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, 4))
        target_seq[0, 0, sampled_char] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [57]:
from tqdm import tqdm
pred = np.zeros((100, x_train.shape[1]))
for seq_index in tqdm(range(100)):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = x_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    pred[seq_index] = decoded_sentence[:-1]

100%|██████████| 100/100 [00:06<00:00, 14.76it/s]


In [58]:
(15/16)**n

0.7241964340209961

In [51]:
qs = Quantum_Strategy(n)
np.mean(qs.check_input_output(np.argmax(x_test[:100], axis=-1), pred[:100]))

0.844

In [59]:
qs = Quantum_Strategy(n)
np.mean(qs.check_input_output(np.argmax(x_test[:100], axis=-1), pred[:100]))

0.842